Yelp es un sitio donde se pueden dejar reseñas y tips sobre negocios de todo tipo, muy utilizado en Estados Unidos. Los csvs son:

* businesses.csv
  * business_id
  * name
  * address
  * city
  * state
  * postal_code
  * latitude
  * longitude
  * stars: float, star rating, rounded to half-stars
  * review_count
  * is_open
  * attributes: un json con atributos variados
  * categories: lista de categorías en json a las que pertenece el negocio
  * hours: diccionario json con horarios día por día en los que abre
* review.csv
  * review_id
  * user_id
  * business_id
  * stars: entero entre 1 y 5
  * date
  * text: texto de la review
  * useful: cantidad de gente que la voto útil
  * funny: cantidad de gente que la voto divertida
  * cool: cantidad de gente que la votó como cool
* user.csv
  * user_id
  * name
  * review_count
  * yelping_since: fecha de registro
  * friends: lista en json de user ids que son amigos
  * useful: cantidad de votos “useful” que recibió el usuario
  * funny: cantidad de votos “funny” que recibió el usuario
  * cool: cantidad de votos “cool” que recibió el usuario
  * fans: cantidad de fans que tiene el usuario
  * elite: lista json con los años que el usuario fue élite
  * average_stars: promedio de rating de todas sus reviews
  * compliment_*: la cantidad de cumplidos de tipo * que recibió el usuario
* checkins.csv: Las visitas que los usuarios registran a los lugares
  * business_id
  * date: lista de fechas separadas por coma de los chekins que recibió el negocio
* tip.csv: tips que los usuarios dan sobre lugares
  * text
  * date
  * compliment_count: cumplidos que recibió el tip
  * business_id
  * user_id

### N4. ¿Cuál es el la review con menor ratio de stopwords en función de su longitud para aquellas que tienen por lo menos 100 caracteres?

In [3]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
reviews = pd.read_csv('/content/drive/MyDrive/Datos/TP1/Parte 1/review.csv', usecols=['text']).text.to_list()

Tokenizo las reviews para calcular su ratio de stopwords como la cantidad de sus tokens que son stopwords / la cantidad total de tokens.

In [5]:
sws = set(stopwords.words())
(min_review, min_ratio) = ('', 1)

for review in reviews:
  if len(review) < 100:
    continue

  review_tokens = word_tokenize(review)
  review_stopwords = [w for w in review_tokens if w in sws]
  ratio = len(review_stopwords) / len(review_tokens)

  if ratio < min_ratio:
    (min_review, min_ratio) = (review, ratio)
  
  if min_ratio == 0: # menor ratio posible
    break

In [6]:
print("El menor ratio de stopwords es " + str(min_ratio) + " para la review: '" + min_review+ "'")

El menor ratio de stopwords es 0.0 para la review: 'Above average pub grub, live music, low key neighborhood bar.  Clean, nice architectural features, nice staff.'


Se encontró una review con ratio de stopwords 0, lo que significa que no contiene ninguna stopword. 

### N8. Queremos crear nuestro propio clasificador de reviews según sean positivas o negativas usando los datos de yelp, para hacer esto vamos a hacer una cosa muy sencilla: asignarle a cada palabra de las 500 más comunes sin contar stopwords el promedio de las stars para las reviews en las que aparece, luego, cuando aparezca un nuevo texto para las palabras que conozcamos del mismo promediamos sus scores. Por ejemplo, si tenemos las palabras “buena” con polaridad 3.4 y “rica” con polaridad 4.3 y tenemos el texto “buena y rica” su predicción será 3.85. Puede usar una muestra para entrenar. ¿Cuál es la salida del predictor para “I loved this place, the food was amazing!”?

In [7]:
import pandas as pd
import numpy as np
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
df = pd.read_csv('/content/drive/MyDrive/Datos/TP1/Parte 1/review.csv', usecols=['text', 'stars'])
df

,stars,text
0,3.0,"If you decide to eat here, just be aware it is..."
1,4.0,Cute interior and owner (?) gave us tour of up...
2,4.0,I was really between 3 and 4 stars for this on...
3,5.0,First time there and it was excellent!!! It fe...
4,5.0,"Great burgers,fries and salad! Burgers have a..."
...,...,...
3838100,5.0,We redesigned my moms dress and mad it complet...
3838101,5.0,"This spot offers a great, affordable east week..."
3838102,4.0,This Home Depot won me over when I needed to g...
3838103,5.0,For when I'm feeling like ignoring my calorie-...


Usando el método Count Vectorizer se pueden obtener las 500 palabras más comunes sin contar stopwords (features), además contar la cantidad de veces que aparecen estos features en cada review.

In [9]:
sws = set(stopwords.words('english'))
count_vectorizer = CountVectorizer(lowercase=True, stop_words=sws, max_features=500)
count_vectorizer.fit_transform(df.text)
features = count_vectorizer.get_feature_names_out()
features[:100]

array(['10', '15', '20', '30', 'able', 'absolutely', 'actually', 'add',
       'ago', 'almost', 'already', 'also', 'although', 'always',
       'amazing', 'amount', 'another', 'anyone', 'anything', 'appetizer',
       'appointment', 'area', 'around', 'arrived', 'ask', 'asked', 'ate',
       'atmosphere', 'attentive', 'available', 'away', 'awesome', 'back',
       'bacon', 'bad', 'bar', 'bartender', 'bbq', 'beans', 'beautiful',
       'beef', 'beer', 'believe', 'best', 'better', 'big', 'bill',
       'birthday', 'bit', 'bowl', 'bread', 'breakfast', 'bring',
       'brought', 'brunch', 'burger', 'business', 'busy', 'buy', 'cake',
       'call', 'called', 'came', 'car', 'card', 'care', 'change',
       'charge', 'check', 'cheese', 'chicken', 'chips', 'chocolate',
       'city', 'clean', 'close', 'coffee', 'cold', 'come', 'comes',
       'comfortable', 'coming', 'company', 'completely', 'cooked', 'cool',
       'could', 'counter', 'couple', 'course', 'covid', 'crab', 'cream',
       'crisp

Calculo las stars promedio de cada feature en un array `feature_stars`, de forma que `feature_stars[x]` es el promedio de stars de `feature[x]`.

In [10]:
feature_stars = []

for feature in features:
  query = count_vectorizer.transform([feature])
  star_sum = 0
  for i in query.indices:
    star_sum += df.stars[i]
  feature_stars.append(star_sum / len(query.indices))

Para predecir las stars de un texto se promedian las stars de cada feature que el texto contenga:

In [11]:
def predict_stars(text):
  query = count_vectorizer.transform([text])
  star_sum = 0
  print("Feature \t Average review stars")
  print("-------------------------------------")
  
  for i in query.indices:
    star_sum += feature_stars[i]
    print(features[i] + "    \t\t " + str(feature_stars[i]))

  prediction = star_sum / len(query.indices)
  print("\nPredicted stars for text '" + text + "': " + str(prediction))
  return prediction

In [12]:
prediction = predict_stars("I loved this place, the food was amazing!")

Feature 	 Average review stars
-------------------------------------
amazing    		 5.0
food    		 5.0
loved    		 5.0
place    		 4.0

Predicted stars for text 'I loved this place, the food was amazing!': 4.75


Se observa que el predictor acertó que una review muy positiva tiene una cantida de estrellas alta.